In [ ]:
import pandas as pd
import geoplot as gplt
import numpy as np
import os
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt
import geopandas as gpd
import pycountry
import descartes
import contextily as ctx
from shapely.geometry import Point, Polygon

In [ ]:
df = pd.read_csv("./dataset_incendios.csv", delimiter=";", decimal=",", encoding="iso-8859-1")
df['provincia_id_fixed'] = [str(x).zfill(2) for x in df['provincia_id']]
df['depar_id_temp'] = [str(x).zfill(3) for x in df['departamento_id']]
df['departamento_id_fixed'] = [prov_id + dept_id for prov_id, dept_id in zip([x for x in df['provincia_id_fixed']], [y for y in df['depar_id_temp']])]
new_cols = list(set(df.columns) - set(['depar_id_temp', 'provincia_id', 'departamento_id']))
df = df[new_cols]
geo_provinces = gpd.read_file('./provincias/provincias.shp')
geo_departments = gpd.read_file('./departamentos/departamentos.shp')
geo_departments_centroid = gpd.read_file('./departamentos/departamentos.shp')
geo_departments_centroid['geometry'] = geo_departments_centroid['geometry'].centroid

In [ ]:
#fires_by_province = df.groupby("provincia_id").size().reset_index(name='counts')
fires_by_province = df.groupby("provincia_id_fixed")['cant_focos'].agg('sum').reset_index(name='counts')
geo_provinces_to_plot = pd.merge(geo_provinces, fires_by_province, left_on='IN1', right_on='provincia_id_fixed', how='outer')
geo_provinces_to_plot = geo_provinces_to_plot[list(['NAM', 'IN1', 'geometry', 'counts'])].fillna(0)
geo_provinces_to_plot['rank'] = geo_provinces_to_plot['counts'].rank()

In [ ]:
sup_by_department = df.groupby('departamento_id_fixed')['sup_afectada'].agg('sum').reset_index(name='counts')
geo_departments_to_plot = pd.merge(geo_departments_centroid, sup_by_department, left_on='IN1', right_on='departamento_id_fixed', how='inner').fillna(0)
geo_departments_to_plot['rank'] = geo_departments_to_plot['counts'].rank()

In [ ]:
ax = gplt.pointplot(geo_departments_to_plot, scale='counts', hue='counts',
                    scheme='EqualInterval', legend=True, legend_var='hue',
                    limits=(2, 20), figsize=(15, 15), cmap='Reds',
                    legend_kwargs={'loc': 'upper right'})
ax.set_title("Hectáreas afectas en incendios forestales (2011 - 2016)", size=20)
gplt.polyplot(geo_provinces_to_plot, ax=ax, edgecolor='white', facecolor='gray', extent=[-74.0, -57.0, -50.0, -21.0])
plt.savefig('./hectareas_afectadas.png')
plt.box(on=None)

In [ ]:
ax = gplt.choropleth(geo_provinces_to_plot, hue='rank', scheme='EqualInterval',
                    legend=True, figsize=(15, 15), cmap='Reds',
                    legend_kwargs={'loc': 'upper right'})
ax.set_title("Focos de incendios forestales (2011 - 2016)", size=20)
gplt.polyplot(geo_provinces_to_plot, ax=ax, extent=[-74.0, -57.0, -50.0, -21.0])
plt.savefig('./focos_totales.png')
plt.box(on=None)